In this notebook we will use the nn class **Sequential** to model a CNN

We will use the single target network

# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [79]:
train_dataset,target=load_script.load_dataset(train=True)

In [80]:
test_dataset,test_target=load_script.load_dataset(train=False)

In [124]:
target=target.long()
test_target=test_target.long()

# Added modules

In [81]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [82]:
class Tensor4D(nn.Module):
    def forward(self,input):
        return input.view(input.shape[0],1,*input.shape[1:])

In [106]:
class Tensor3D(nn.Module):
    def forward(self,input):
        return input.view(input.shape[0],input.shape[1],-1)

# Normalization

In [107]:
mean.shape

torch.Size([1, 28, 50])

In [108]:
train_dataset.mean(0)

Variable containing:
1.00000e-08 *
 1.4736 -0.1368  0.7067  ...   0.3584  0.6602  4.5081
-0.3961  0.6036  0.6602  ...  -0.2263  1.0563 -3.4329
 1.5844  0.3018 -3.0180  ...  -3.3575  2.1880 -0.9054
          ...             ⋱             ...          
 2.0371  0.7545  0.7545  ...   0.4150  1.1317  0.6790
 0.6036  0.0754  0.9054  ...   1.3581 -0.7545 -0.6790
-0.6036  1.5090 -2.1126  ...  -0.7922 -1.6599  0.8299
[torch.FloatTensor of size 28x50]

In [109]:
mean=train_dataset.mean(0).view(1,28,-1)
std=train_dataset.std(0).view(1,28,-1)

In [110]:
train_dataset=(train_dataset-mean)/std
test_dataset=(test_dataset-mean)/std

# Defining the network

### Single output

In [111]:
class SingleOutput(nn.Module):
    def __init__(self):
        super(SingleOutput,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

### Sequential

In [112]:
# 50 -> 48 -> 24 -> 20 -> 10

In [141]:
class Sequential(nn.Module):
    def __init__(self):
        super(Sequential,self).__init__()
        
        layers=[]
        layers.append(nn.Conv1d(28,28,3))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        layers.append(nn.Conv1d(28,28,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        layers.append(nn.Conv1d(28,28,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        layers.append(Flatten())
        layers.append(nn.Linear(28*3,200))
        layers.append(nn.ReLU())
#         layers.append(nn.Linear(200,20))
#         layers.append(nn.ReLU())
        layers.append(nn.Linear(200,2))
#         layers.append(nn.Sigmoid())
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

### Convolution 2d

In [147]:
class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D,self).__init__()
        
        # 28*50
        layers=[]
        layers.append(nn.Conv1d(28,28,3))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        
        # 28*24
        layers.append(nn.Conv1d(28,28,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        
        # 28*10
        layers.append(Tensor4D())
        layers.append(nn.Conv2d(1,10,5))
        # 10*24*6
        layers.append(Tensor3D())
        # 10*144
        
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        # 10*72
        
        layers.append(Flatten())
        layers.append(nn.Linear(10*72,200))
        layers.append(nn.ReLU())
        
        layers.append(nn.Linear(200,20))
        layers.append(nn.ReLU())
        
        layers.append(nn.Linear(20,2))        
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

# Training

In [148]:
net=CNN2D()

optimizer = optim.SGD(net.parameters(), lr=1)
optimizer = optim.Adam(net.parameters(), lr=0.0001,weight_decay=0)


criterion=nn.CrossEntropyLoss()
mini_batch_size=79

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        

        out=net(train_element)
        output_target[b:b+mini_batch_size]=(out[:,1]>out[:,0]).data
        
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    out=net(test_dataset)
    output_test=(out[:,1]>out[:,0]).data.long()
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target.long()==target.data))/target.shape[0]
    error_test=np.sum(list(output_test==test_target.data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

Epoch: 0 Loss: 2.7767593264579773 Correct: 49.68% Correct test: 51.0%
Epoch: 10 Loss: 2.7660399079322815 Correct: 50.63% Correct test: 49.0%
Epoch: 20 Loss: 2.746740996837616 Correct: 63.60% Correct test: 49.0%
Epoch: 30 Loss: 2.6985633969306946 Correct: 62.65% Correct test: 47.0%
Epoch: 40 Loss: 2.6030282974243164 Correct: 66.77% Correct test: 48.0%
Epoch: 50 Loss: 2.4674468636512756 Correct: 69.62% Correct test: 50.0%
Epoch: 60 Loss: 2.2806867361068726 Correct: 74.68% Correct test: 54.0%
Epoch: 70 Loss: 2.0417137444019318 Correct: 77.53% Correct test: 60.0%
Epoch: 80 Loss: 1.803324967622757 Correct: 81.64% Correct test: 59.0%
Epoch: 90 Loss: 1.6020318865776062 Correct: 84.81% Correct test: 60.0%
Epoch: 100 Loss: 1.427435278892517 Correct: 87.65% Correct test: 61.0%
Epoch: 110 Loss: 1.2746775448322296 Correct: 89.55% Correct test: 63.0%
Epoch: 120 Loss: 1.133047878742218 Correct: 91.45% Correct test: 63.0%
Epoch: 130 Loss: 0.9954690784215927 Correct: 91.77% Correct test: 62.0%
Epoch: 

KeyboardInterrupt: 

In [ ]:
79*720

In [105]:
360*200

72000